In [43]:
import pandas as pd
import geopandas as gpd
import numpy as np


boston_listings = pd.read_csv("boston_listings.csv")
cambridge_listings = pd.read_csv("cambridge_listings.csv")

boston_listings = gpd.GeoDataFrame(boston_listings, geometry=gpd.points_from_xy(boston_listings["longitude"], boston_listings["latitude"]), crs="EPSG:4326").to_crs("EPSG:26986")
cambridge_listings = gpd.GeoDataFrame(cambridge_listings, geometry=gpd.points_from_xy(cambridge_listings["longitude"], cambridge_listings["latitude"]), crs="EPSG:4326").to_crs("EPSG:26986")
all_listings = pd.concat([boston_listings,cambridge_listings])

station_areas = pd.read_csv("../stations_routes_f23_geoms_nonoverlapping.csv")

station_areas= gpd.GeoDataFrame(station_areas,geometry=gpd.GeoSeries.from_wkt(station_areas["walk_poly_wkt"]))
station_areas = station_areas.set_crs("EPSG:4326").to_crs("EPSG:26986")

In [44]:
points_in_polygons = gpd.sjoin(all_listings, station_areas, how="inner", predicate="within")

polygon_point_counts = points_in_polygons.groupby(["route_id","parent_station"]).size().reset_index(name="airbnb_count")

station_areas=station_areas.merge(polygon_point_counts,on=["route_id","parent_station"],how="left")


station_areas["acreage"] = station_areas.geometry.area / 4046.86

# station_areas["airbnb_density"] = station_areas["airbnb_count"].fillna(0).astype(int) / station_areas["acreage"].replace(0, float("nan"))


station_areas["airbnb_density"] = np.where(
    station_areas["airbnb_count"].isna(),
    np.nan,  # Make null if point_count is null
    station_areas["airbnb_count"] / station_areas["acreage"].replace(0, np.nan)
)


In [46]:
station_areas[["route_id","parent_station","airbnb_count","airbnb_density"]].to_csv("stations_routes_airbnbs_2024_bos_cam.csv")